# 使用序列到序列模型完成数字加法
* 作者：[jm12138](https://github.com/jm12138)
* 日期：2020.10.21

## 简要介绍
* 本示例教程介绍如何使用飞桨完成一个数字加法任务
* 我们将会使用飞桨提供的LSTM的API，组建一个序列到序列模型
* 并在随机生成的数据集上完成数字加法任务的模型训练与预测

## 环境设置
* 本示例教程基于飞桨2.0-rc版本

In [1]:
# 导入项目运行所需的包
import random
import numpy as np

import paddle
import paddle.nn as nn

from visualdl import LogWriter

# 打印Paddle版本
print('paddle version: %s' % paddle.__version__)

# 设置CPU为运行位置
place = paddle.CPUPlace()

paddle version: 2.0.0-rc0


## 构建数据集
* 随机生成数据，并使用生成的数据构造数据集
* 通过继承paddle.io.Dataset来完成数据集的构造

In [8]:
# 编码函数
def encoder(text, LEN, label_dict):
    # 文本转ID
    ids = [label_dict[word] for word in text]
    # 对长度进行补齐
    ids += [label_dict[' ']]*(LEN-len(ids))
    return ids

# 单个数据生成函数
def make_data(inputs, labels, DIGITS, label_dict):
    MAXLEN = DIGITS + 1 + DIGITS
    # 对输入输出文本进行ID编码
    inputs = encoder(inputs, MAXLEN, label_dict)
    labels = encoder(labels, DIGITS + 1, label_dict)
    return inputs, labels

# 批量数据生成函数
def gen_datas(DATA_NUM, MAX_NUM, DIGITS, label_dict):
    datas = []
    while len(datas)<DATA_NUM:
        # 随机取两个数
        a = random.randint(0,MAX_NUM)
        b = random.randint(0,MAX_NUM)
        # 生成输入文本
        inputs = '%d+%d' % (a, b)
        # 生成输出文本
        labels = str(eval(inputs))
        # 生成单个数据
        inputs, labels = [np.array(_).astype('int64') for _ in make_data(inputs, labels, DIGITS, label_dict)]
        datas.append([inputs, labels])
    return datas

# 继承paddle.io.Dataset来构造数据集
class Addition_Dataset(paddle.io.Dataset):
    # 重写数据集初始化函数
    def __init__(self, datas):
        super(Addition_Dataset, self).__init__()
        self.datas = datas
    
    # 重写生成样本的函数
    def __getitem__(self, index):
        data, label = [paddle.to_tensor(_) for _ in self.datas[index]]
        return data, label

    # 重写返回数据集大小的函数
    def __len__(self):
        return len(self.datas)

print('generating datas..')

# 定义字符表
label_dict = {
    '0': 0, '1': 1, '2': 2, '3': 3,
    '4': 4, '5': 5, '6': 6, '7': 7,
    '8': 8, '9': 9, '+': 10, ' ': 11
}

# 输入数字最大位数
DIGITS = 2

# 数据数量
train_num = 5000
dev_num = 500

# 数据批大小
batch_size = 32

# 读取线程数
num_workers = 8

# 定义一些所需变量
MAXLEN = DIGITS + 1 + DIGITS
MAX_NUM = 10**(DIGITS)-1

# 生成数据
train_datas = gen_datas(
    train_num, 
    MAX_NUM,
    DIGITS, 
    label_dict
) 
dev_datas = gen_datas(
    dev_num, 
    MAX_NUM,
    DIGITS, 
    label_dict
)

# 实例化数据集
train_dataset = Addition_Dataset(train_datas)
dev_dataset = Addition_Dataset(dev_datas)

print('making the dataset...')

# 实例化数据读取器
train_reader = paddle.io.DataLoader(
    train_dataset,
    places=place,
    batch_size=batch_size,
    shuffle=True,
    drop_last=False,
    num_workers=num_workers
)
dev_reader = paddle.io.DataLoader(
    dev_dataset,
    places=place,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

print('finish')

generating datas..
making the dataset...
finish


##  模型组网
* 通过继承paddle.nn.Layer类来搭建模型
* 本次介绍的模型是一个简单的基于LSTM的Seq2Seq模型
* 一共有如下四个主要的网络层：

  1. 嵌入层(Embedding)：将输入的文本序列转为嵌入向量
  2. 编码层(LSTM)：将嵌入向量进行编码
  3. 解码层(LSTM)：将编码向量进行解码
  4. 全连接层(Linear)：对解码完成的向量进行线性映射
* 损失函数为交叉熵损失函数

In [3]:
# 继承paddle.nn.Layer类
class Addition_Model(nn.Layer):
    # 重写初始化函数
    # 参数：字符表长度、嵌入层大小、隐藏层大小、解码器层数、处理数字的最大位数
    def __init__(self, char_len=12, embedding_size=128, hidden_size=128, num_layers=1, DIGITS=2):
        super(Addition_Model, self).__init__()
        # 初始化变量
        self.DIGITS = DIGITS
        self.MAXLEN = DIGITS + 1 + DIGITS
        self.hidden_size = hidden_size
        self.char_len = char_len

        # 嵌入层
        self.emb = nn.Embedding(
            char_len, 
            embedding_size
        )
        
        # 编码器
        self.encoder = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=1
        )
        
        # 解码器
        self.decoder = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers
        )
        
        # 全连接层
        self.fc = nn.Linear(
            hidden_size, 
            char_len
        )
    
    # 重写模型前向计算函数
    # 参数：输入[None, MAXLEN]、标签[None, DIGITS + 1]
    def forward(self, inputs, labels=None):
        # 嵌入层
        out = self.emb(inputs)

        # 编码器
        out, (_, _) = self.encoder(out)

        # 按时间步切分编码器输出
        out = paddle.split(out, self.MAXLEN, axis=1)

        # 取最后一个时间步的输出并复制 DIGITS + 1 次
        out = paddle.expand(out[-1], [out[-1].shape[0], self.DIGITS + 1, self.hidden_size])

        # 解码器
        out, (_, _) = self.decoder(out)

        # 全连接
        out = self.fc(out)

        # 如果标签存在，则计算其损失和准确率
        if labels is not None:
            # 转置解码器输出
            tmp = paddle.transpose(out, [0, 2, 1])

            # 计算交叉熵损失
            loss = nn.functional.cross_entropy(tmp, labels)

            # 计算准确率
            acc = paddle.metric.accuracy(paddle.reshape(out, [-1, self.char_len]), paddle.reshape(labels, [-1, 1]))

            # 返回损失和准确率
            return loss, acc

        # 返回输出
        return out

## 模型训练与评估
* 使用Adam作为优化器进行模型训练
* 以模型准确率作为评价指标
* 使用VisualDL对训练数据进行可视化
* 训练过程中会同时进行模型评估和最佳模型的保存

In [9]:
# 初始化log写入器
log_writer = LogWriter(logdir="./log")

# 模型参数设置
embedding_size = 128
hidden_size=128
num_layers=1

# 训练参数设置
epoch_num = 200
learning_rate = 0.001
log_iter = 20
eval_iter = 500

# 定义一些所需变量
global_step = 0
log_step = 0
max_acc = 0

# 实例化模型
model = Addition_Model(
    char_len=len(label_dict), 
    embedding_size=embedding_size, 
    hidden_size=hidden_size, 
    num_layers=num_layers, 
    DIGITS=DIGITS)

# 将模型设置为训练模式
model.train()

# 设置优化器，学习率，并且把模型参数给优化器
opt = paddle.optimizer.Adam(
    learning_rate=learning_rate,
    parameters=model.parameters()
)

# 启动训练，循环epoch_num个轮次
for epoch in range(epoch_num):
    # 遍历数据集读取数据
    for batch_id, data in enumerate(train_reader()):
        # 读取数据
        inputs, labels = data

        # 模型前向计算
        loss, acc = model(inputs, labels=labels)

        # 打印训练数据
        if global_step%log_iter==0:
            print('train epoch:%d step: %d loss:%f acc:%f' % (epoch, global_step, loss.numpy(), acc.numpy()))
            log_writer.add_scalar(tag="train/loss", step=log_step, value=loss.numpy())
            log_writer.add_scalar(tag="train/acc", step=log_step, value=acc.numpy())
            log_step+=1

        # 模型验证
        if global_step%eval_iter==0:
            model.eval()
            losses = []
            accs = []
            for data in dev_reader():
                loss, acc = model(inputs, labels=labels)
                losses.append(loss.numpy())
                accs.append(acc.numpy())
            avg_loss = np.concatenate(losses).mean()
            avg_acc = np.concatenate(accs).mean()
            print('eval epoch:%d step: %d loss:%f acc:%f' % (epoch, global_step, avg_loss, avg_acc))
            log_writer.add_scalar(tag="dev/loss", step=log_step, value=avg_loss)
            log_writer.add_scalar(tag="dev/acc", step=log_step, value=avg_acc)

            # 保存最佳模型
            if avg_acc>max_acc:
                max_acc = avg_acc
                print('saving the best_model...')
                paddle.save(model.state_dict(), 'best_model')
            model.train()

        # 反向传播
        loss.backward()

        # 使用优化器进行参数优化
        opt.step()

        # 清除梯度
        opt.clear_grad()

        # 全局步数加一
        global_step += 1

# 保存最终模型
paddle.save(model.state_dict(),'final_model')

train epoch:0 step: 0 loss:2.485033 acc:0.093750
eval epoch:0 step: 0 loss:2.485033 acc:0.093750
saving the best_model...
train epoch:0 step: 20 loss:2.408553 acc:0.145833
train epoch:0 step: 40 loss:2.235319 acc:0.395833
train epoch:0 step: 60 loss:2.215429 acc:0.406250
train epoch:0 step: 80 loss:2.252749 acc:0.354167
train epoch:0 step: 100 loss:2.244046 acc:0.375000
train epoch:0 step: 120 loss:2.253286 acc:0.364583
train epoch:0 step: 140 loss:2.224211 acc:0.395833
train epoch:1 step: 160 loss:2.254224 acc:0.364583
train epoch:1 step: 180 loss:2.253024 acc:0.364583
train epoch:1 step: 200 loss:2.253495 acc:0.364583
train epoch:1 step: 220 loss:2.150470 acc:0.468750
train epoch:1 step: 240 loss:2.262765 acc:0.354167
train epoch:1 step: 260 loss:2.253251 acc:0.364583
train epoch:1 step: 280 loss:2.214941 acc:0.406250
train epoch:1 step: 300 loss:2.241283 acc:0.375000
train epoch:2 step: 320 loss:2.252329 acc:0.364583
train epoch:2 step: 340 loss:2.237661 acc:0.385417
train epoch:2 s

train epoch:19 step: 3000 loss:2.088592 acc:0.510417
eval epoch:19 step: 3000 loss:2.088592 acc:0.510417
train epoch:19 step: 3020 loss:2.102115 acc:0.520833
train epoch:19 step: 3040 loss:2.054089 acc:0.562500
train epoch:19 step: 3060 loss:2.069692 acc:0.552083
train epoch:19 step: 3080 loss:2.020173 acc:0.614583
train epoch:19 step: 3100 loss:2.111103 acc:0.510417
train epoch:19 step: 3120 loss:2.011185 acc:0.614583
train epoch:20 step: 3140 loss:1.972947 acc:0.645833
train epoch:20 step: 3160 loss:2.110248 acc:0.510417
train epoch:20 step: 3180 loss:2.110843 acc:0.500000
train epoch:20 step: 3200 loss:2.045410 acc:0.572917
train epoch:20 step: 3220 loss:2.061454 acc:0.552083
train epoch:20 step: 3240 loss:2.116978 acc:0.479167
train epoch:20 step: 3260 loss:2.075301 acc:0.562500
train epoch:20 step: 3280 loss:2.020166 acc:0.593750
train epoch:21 step: 3300 loss:2.028885 acc:0.604167
train epoch:21 step: 3320 loss:2.012455 acc:0.604167
train epoch:21 step: 3340 loss:2.050273 acc:0.5

train epoch:37 step: 5940 loss:1.838856 acc:0.781250
train epoch:37 step: 5960 loss:1.774894 acc:0.854167
train epoch:38 step: 5980 loss:1.713140 acc:0.916667
train epoch:38 step: 6000 loss:1.735639 acc:0.885417
eval epoch:38 step: 6000 loss:1.735639 acc:0.885417
saving the best_model...
train epoch:38 step: 6020 loss:1.829078 acc:0.791667
train epoch:38 step: 6040 loss:1.742466 acc:0.885417
train epoch:38 step: 6060 loss:1.792089 acc:0.833333
train epoch:38 step: 6080 loss:1.728374 acc:0.895833
train epoch:38 step: 6100 loss:1.739469 acc:0.885417
train epoch:38 step: 6120 loss:1.727779 acc:0.906250
train epoch:39 step: 6140 loss:1.767244 acc:0.864583
train epoch:39 step: 6160 loss:1.806533 acc:0.822917
train epoch:39 step: 6180 loss:1.800849 acc:0.812500
train epoch:39 step: 6200 loss:1.792199 acc:0.833333
train epoch:39 step: 6220 loss:1.751119 acc:0.875000
train epoch:39 step: 6240 loss:1.767042 acc:0.864583
train epoch:39 step: 6260 loss:1.811072 acc:0.802083
train epoch:40 step: 6

train epoch:56 step: 8900 loss:1.664687 acc:0.958333
train epoch:56 step: 8920 loss:1.716503 acc:0.906250
train epoch:56 step: 8940 loss:1.715070 acc:0.906250
train epoch:57 step: 8960 loss:1.725543 acc:0.895833
train epoch:57 step: 8980 loss:1.709907 acc:0.916667
train epoch:57 step: 9000 loss:1.704064 acc:0.916667
eval epoch:57 step: 9000 loss:1.704064 acc:0.916667
train epoch:57 step: 9020 loss:1.651235 acc:0.968750
train epoch:57 step: 9040 loss:1.685288 acc:0.937500
train epoch:57 step: 9060 loss:1.719383 acc:0.906250
train epoch:57 step: 9080 loss:1.713328 acc:0.906250
train epoch:57 step: 9100 loss:1.699857 acc:0.927083
train epoch:58 step: 9120 loss:1.665797 acc:0.958333
train epoch:58 step: 9140 loss:1.727639 acc:0.885417
train epoch:58 step: 9160 loss:1.686109 acc:0.937500
train epoch:58 step: 9180 loss:1.689479 acc:0.927083
train epoch:58 step: 9200 loss:1.716828 acc:0.916667
train epoch:58 step: 9220 loss:1.664574 acc:0.958333
train epoch:58 step: 9240 loss:1.644176 acc:0.9

train epoch:75 step: 11820 loss:1.659688 acc:0.968750
train epoch:75 step: 11840 loss:1.663348 acc:0.958333
train epoch:75 step: 11860 loss:1.682247 acc:0.937500
train epoch:75 step: 11880 loss:1.641298 acc:0.979167
train epoch:75 step: 11900 loss:1.689550 acc:0.927083
train epoch:75 step: 11920 loss:1.689580 acc:0.927083
train epoch:76 step: 11940 loss:1.649229 acc:0.979167
train epoch:76 step: 11960 loss:1.633778 acc:0.989583
train epoch:76 step: 11980 loss:1.697053 acc:0.927083
train epoch:76 step: 12000 loss:1.660124 acc:0.958333
eval epoch:76 step: 12000 loss:1.660124 acc:0.958333
train epoch:76 step: 12020 loss:1.690063 acc:0.927083
train epoch:76 step: 12040 loss:1.663207 acc:0.958333
train epoch:76 step: 12060 loss:1.651356 acc:0.968750
train epoch:76 step: 12080 loss:1.671807 acc:0.947917
train epoch:77 step: 12100 loss:1.621480 acc:1.000000
train epoch:77 step: 12120 loss:1.640068 acc:0.979167
train epoch:77 step: 12140 loss:1.669735 acc:0.947917
train epoch:77 step: 12160 lo

...

train epoch:112 step: 17620 loss:1.651245 acc:0.968750
train epoch:112 step: 17640 loss:1.636205 acc:0.979167
train epoch:112 step: 17660 loss:1.670953 acc:0.947917
train epoch:112 step: 17680 loss:1.702054 acc:0.916667
train epoch:112 step: 17700 loss:1.683097 acc:0.937500
train epoch:112 step: 17720 loss:1.681103 acc:0.937500
train epoch:112 step: 17740 loss:1.660480 acc:0.958333
train epoch:113 step: 17760 loss:1.650308 acc:0.968750
train epoch:113 step: 17780 loss:1.629344 acc:0.989583
train epoch:113 step: 17800 loss:1.660249 acc:0.958333
train epoch:113 step: 17820 loss:1.650166 acc:0.968750
train epoch:113 step: 17840 loss:1.650404 acc:0.968750
train epoch:113 step: 17860 loss:1.649828 acc:0.968750
train epoch:113 step: 17880 loss:1.670738 acc:0.947917
train epoch:114 step: 17900 loss:1.650108 acc:0.968750
train epoch:114 step: 17920 loss:1.629312 acc:0.989583
train epoch:114 step: 17940 loss:1.660625 acc:0.958333
train epoch:114 step: 17960 loss:1.680867 acc:0.937500
train epoc

train epoch:130 step: 20520 loss:1.649656 acc:0.968750
train epoch:130 step: 20540 loss:1.650017 acc:0.968750
train epoch:130 step: 20560 loss:1.680748 acc:0.937500
train epoch:131 step: 20580 loss:1.660443 acc:0.958333
train epoch:131 step: 20600 loss:1.681270 acc:0.937500
train epoch:131 step: 20620 loss:1.650050 acc:0.968750
train epoch:131 step: 20640 loss:1.629179 acc:0.989583
train epoch:131 step: 20660 loss:1.629203 acc:0.989583
train epoch:131 step: 20680 loss:1.680635 acc:0.937500
train epoch:131 step: 20700 loss:1.649745 acc:0.968750
train epoch:131 step: 20720 loss:1.649654 acc:0.968750
train epoch:132 step: 20740 loss:1.639623 acc:0.979167
train epoch:132 step: 20760 loss:1.649382 acc:0.968750
train epoch:132 step: 20780 loss:1.670857 acc:0.947917
train epoch:132 step: 20800 loss:1.649679 acc:0.968750
train epoch:132 step: 20820 loss:1.680942 acc:0.937500
train epoch:132 step: 20840 loss:1.639284 acc:0.979167
train epoch:132 step: 20860 loss:1.639615 acc:0.979167
train epoc

train epoch:149 step: 23420 loss:1.650252 acc:0.968750
train epoch:149 step: 23440 loss:1.660205 acc:0.958333
train epoch:149 step: 23460 loss:1.647797 acc:0.968750
train epoch:149 step: 23480 loss:1.619283 acc:1.000000
train epoch:149 step: 23500 loss:1.681245 acc:0.937500
eval epoch:149 step: 23500 loss:1.681245 acc:0.937500
train epoch:149 step: 23520 loss:1.660530 acc:0.958333
train epoch:149 step: 23540 loss:1.649601 acc:0.968750
train epoch:150 step: 23560 loss:1.639684 acc:0.979167
train epoch:150 step: 23580 loss:1.670917 acc:0.947917
train epoch:150 step: 23600 loss:1.660176 acc:0.958333
train epoch:150 step: 23620 loss:1.660618 acc:0.958333
train epoch:150 step: 23640 loss:1.681171 acc:0.937500
train epoch:150 step: 23660 loss:1.640629 acc:0.979167
train epoch:150 step: 23680 loss:1.649564 acc:0.968750
train epoch:150 step: 23700 loss:1.661741 acc:0.958333
train epoch:151 step: 23720 loss:1.639711 acc:0.979167
train epoch:151 step: 23740 loss:1.639324 acc:0.979167
train epoch

train epoch:167 step: 26280 loss:1.670848 acc:0.947917
train epoch:167 step: 26300 loss:1.639598 acc:0.979167
train epoch:167 step: 26320 loss:1.639609 acc:0.979167
train epoch:167 step: 26340 loss:1.660442 acc:0.958333
train epoch:167 step: 26360 loss:1.681290 acc:0.937500
train epoch:168 step: 26380 loss:1.649615 acc:0.968750
train epoch:168 step: 26400 loss:1.681271 acc:0.937500
train epoch:168 step: 26420 loss:1.670542 acc:0.947917
train epoch:168 step: 26440 loss:1.691326 acc:0.927083
train epoch:168 step: 26460 loss:1.629228 acc:0.989583
train epoch:168 step: 26480 loss:1.649628 acc:0.968750
train epoch:168 step: 26500 loss:1.618818 acc:1.000000
eval epoch:168 step: 26500 loss:1.618818 acc:1.000000
train epoch:168 step: 26520 loss:1.618778 acc:1.000000
train epoch:169 step: 26540 loss:1.650016 acc:0.968750
train epoch:169 step: 26560 loss:1.660429 acc:0.958333
train epoch:169 step: 26580 loss:1.639184 acc:0.979167
train epoch:169 step: 26600 loss:1.639188 acc:0.979167
train epoch

train epoch:185 step: 29160 loss:1.681282 acc:0.937500
train epoch:185 step: 29180 loss:1.653351 acc:0.968750
train epoch:185 step: 29200 loss:1.656091 acc:0.958333
train epoch:186 step: 29220 loss:1.662224 acc:0.958333
train epoch:186 step: 29240 loss:1.650155 acc:0.968750
train epoch:186 step: 29260 loss:1.660690 acc:0.958333
train epoch:186 step: 29280 loss:1.670634 acc:0.947917
train epoch:186 step: 29300 loss:1.663186 acc:0.958333
train epoch:186 step: 29320 loss:1.642247 acc:0.979167
train epoch:186 step: 29340 loss:1.641601 acc:0.979167
train epoch:187 step: 29360 loss:1.660100 acc:0.958333
train epoch:187 step: 29380 loss:1.640132 acc:0.979167
train epoch:187 step: 29400 loss:1.671129 acc:0.947917
train epoch:187 step: 29420 loss:1.620439 acc:1.000000
train epoch:187 step: 29440 loss:1.689038 acc:0.927083
train epoch:187 step: 29460 loss:1.694560 acc:0.927083
train epoch:187 step: 29480 loss:1.639368 acc:0.979167
train epoch:187 step: 29500 loss:1.687535 acc:0.927083
eval epoch

## 模型测试
* 使用保存的最佳模型进行测试

In [7]:
# 反转字符表
label_dict_adv = {v: k for k, v in label_dict.items()}

# 输入计算题目
input_text = '12+40'

# 编码输入为ID
inputs = encoder(input_text, MAXLEN, label_dict)

# 转换输入为向量形式
inputs = np.array(inputs).reshape(-1, MAXLEN)
inputs = paddle.to_tensor(inputs)

# 加载模型
params_dict= paddle.load('best_model')
model.set_dict(params_dict)

# 设置为评估模式
model.eval()

# 模型推理
out = model(inputs)

# 结果转换
result = ''.join([label_dict_adv[_] for _ in np.argmax(out.numpy(), -1).reshape(-1)])

# 打印结果
print('the model answer: %s=%s' % (input_text, result))
print('the true answer: %s=%s' % (input_text, eval(input_text)))

the model answer: 12+40=52 
the true answer: 12+40=52


## 总结
* 你还可以通过变换网络结构，调整数据集，尝试不同的参数的方式来进一步提升本示例当中的数字加法的效果
* 同时，也可以尝试在其他的类似的任务中用飞桨来完成实际的实践